In [ ]:
# Cài đặt Ollama
!curl -fsSL https://ollama.com/install.sh | sh
# Cài đặt thư viện Python cho giao diện và tunnel
!pip install streamlit pyngrok ollama
# Cài đặt thư viện Firebase (ví dụ: sử dụng firebase_admin cho Python server-side)
!pip install firebase-admin

In [ ]:
import os
env = os.environ.copy()
env["OLLAMA_HOST"] = "0.0.0.0"
env["OLLAMA_ORIGINS"] = "*"

import subprocess
def run_ollama_serve():
  subprocess.Popen(["ollama", "serve"], env=env)

import threading
thread = threading.Thread(target=run_ollama_serve)
thread.start()

import time
time.sleep(5)

In [ ]:
!ollama pull mistral

from pyngrok import ngrok
from google.colab import userdata

# Lấy ngrok API key từ Colab Secrets
ngrok_api_key = userdata.get('ngrok_api_key')
ngrok.set_auth_token(ngrok_api_key)

# Tạo tunnel cho port 11434 (Ollama API)
try:
    ollama_public_url = ngrok.connect(11434, proto="http").public_url
    print(f"Ollama Public URL: {ollama_public_url}")
    # Lưu URL vào session state để dùng cho Streamlit sau
    # (Tạm thời lưu trong biến, sẽ dùng session_state khi chạy Streamlit)
except Exception as e:
    print(f"Lỗi Ngrok: {e}. Đảm bảo Ollama đã chạy và API key đúng.")

In [ ]:
%%writefile app.py
import streamlit as st
import requests
import firebase_admin
from firebase_admin import credentials, auth, firestore

# ==============================================================================
# PHẦN 1: FIREBASE INIT & AUTHENTICATION (SỬA LỖI DB)
# ==============================================================================

# Khởi tạo session state (Quan trọng để tránh lỗi 'has no attribute')
if 'user_logged_in' not in st.session_state:
    st.session_state.user_logged_in = False
if 'db' not in st.session_state:
    st.session_state.db = None # Khởi tạo ban đầu là None

def init_firebase():
    if not st.session_state.db:
        try:
            # THAY 'your-firebase-key.json' bằng tên file key bạn đã upload
            cred = credentials.Certificate("firebase.json")
            firebase_app = firebase_admin.initialize_app(cred)
            st.session_state.db = firestore.client()
            st.success("Kết nối Firebase thành công.")
        except Exception as e:
            st.error(f"Lỗi khởi tạo Firebase: {e}. Đảm bảo file key đã được upload.")
            return False
    return True

# Hàm xử lý Đăng nhập/Đăng ký đã sửa
def authenticate_user(email, password, is_register=False):
    if not init_firebase(): # Đảm bảo Firebase được khởi tạo trước
        return

    try:
        if is_register:
            # Xử lý ĐĂNG KÝ
            user = auth.create_user(email=email, password=password)
            st.session_state.user_id = user.uid
            st.session_state.user_email = email
            st.session_state.user_logged_in = True
            st.success("Đăng ký thành công! Vui lòng Đăng nhập.")

        else:
            # Xử lý ĐĂNG NHẬP (Chỉ kiểm tra sự tồn tại của email)
            user = auth.get_user_by_email(email)
            st.session_state.user_id = user.uid
            st.session_state.user_email = email
            st.session_state.user_logged_in = True
            st.success(f"Chào mừng, {email}!")

    except firebase_admin.exceptions.FirebaseError as fe:
        if 'email-already-exists' in str(fe):
            st.error("Email này đã được đăng ký.")
        elif 'user-not-found' in str(fe) and not is_register:
            st.error("Email chưa được đăng ký. Vui lòng kiểm tra lại hoặc Đăng ký.")
        else:
            st.error(f"Lỗi Firebase: {fe}")
    except Exception as e:
        st.error(f"Lỗi chung: {e}")

# ==============================================================================
# PHẦN 2: LLM LOGIC (Hàm gọi Ollama)
# ==============================================================================
def generate_itinerary(origin, dest, dates, interests, pace, ollama_url):
    interest_str = ", ".join(interests)
    prompt = (
        f"Suggest a day-by-day itinerary (morning/afternoon/evening) with short explanations. "
        f"The trip is from {origin} to {dest}, lasting {dates}. "
        f"The traveler prefers a {pace} pace and is interested in: {interest_str}. "
        f"Please respond only with the itinerary."
    )

    payload = {
        "model": "mistral",
        "prompt": prompt,
        "stream": False
    }

    try:
        response = requests.post(f"{ollama_url}/api/generate", json=payload, timeout=120)
        if response.status_code == 200:
            return response.json()['response']
        else:
            return f"Lỗi gọi LLM Server (Code {response.status_code}): {response.text}"
    except requests.exceptions.RequestException as e:
        return f"Lỗi kết nối Ollama: {e}. Kiểm tra URL Ngrok."

# ==============================================================================
# PHẦN 3: GIAO DIỆN CHÍNH STREAMLIT
# ==============================================================================
st.title("✈️ AI Travel Recommendation Assistant")

if not st.session_state.user_logged_in:
    st.subheader("Đăng nhập/Đăng ký")

    with st.expander("Đăng nhập/Đăng ký"):
        email = st.text_input("Email:")
        password = st.text_input("Mật khẩu:", type="password")

        col1, col2 = st.columns(2)
        with col1:
            if st.button("Đăng nhập"):
                authenticate_user(email, password, is_register=False)
        with col2:
            if st.button("Đăng ký"):
                authenticate_user(email, password, is_register=True)

# --- Giao diện Ứng dụng chính ---
else:
    init_firebase() # Đảm bảo Firebase được khởi tạo

    st.success(f"Xin chào, {st.session_state.user_email}!")

    # Dán Ollama URL MỚI (sau khi chạy lại Cell 3) vào biến này
    ollama_url = "https://interrogatorily-untarnishable-leisha.ngrok-free.dev"
    st.session_state.ollama_url = ollama_url

    tab1, tab2 = st.tabs(["🗺️ Lập Kế hoạch", "📚 Lịch sử"])

    with tab1:
        with st.form("travel_form"):
            st.header("Lên kế hoạch Du lịch Mới")
            # ... (Phần input không thay đổi) ...
            origin_city = st.text_input("Thành phố Xuất phát (Origin City):", "Hà Nội")
            dest_city = st.text_input("Thành phố Đến (Destination City):", "Đà Nẵng")
            dates = st.text_input("Ngày đi (Dates - e.g., 3 ngày, 10-12/12):", "3 ngày")

            interests = st.multiselect(
                "Sở thích (Interests):",
                ['Ẩm thực (Food)', 'Bảo tàng/Văn hóa (Museums)', 'Thiên nhiên (Nature)', 'Giải trí đêm (Nightlife)']
            )

            pace = st.radio(
                "Tốc độ Du lịch (Pace):",
                ['Nhàn nhã (Relaxed)', 'Bình thường (Normal)', 'Gấp gáp (Tight)']
            )

            submitted = st.form_submit_button("Tạo Lịch trình")

        if submitted and dest_city:
            if ollama_url == "DÁN_OLLAMA_PUBLIC_URL_MỚI_VÀO_ĐÂY":
                 st.error("Vui lòng dán Ollama URL đã tạo ở bước 1 vào code app.py.")
            else:
                with st.spinner('⏳ Đang tạo lịch trình... (Có thể mất đến 1 phút)'):
                    itinerary_text = generate_itinerary(origin_city, dest_city, dates, interests, pace, ollama_url)

                    st.subheader(f"✅ Lịch trình cho {dest_city}")
                    st.markdown(itinerary_text)

                    # Lưu vào Chat History/Database
                    if st.session_state.db: # Kiểm tra lần nữa trước khi lưu
                        try:
                            st.session_state.db.collection('itineraries').add({
                                'user_id': st.session_state.user_id,
                                'destination': dest_city,
                                'input': {'origin': origin_city, 'dates': dates, 'interests': interests, 'pace': pace},
                                'itinerary': itinerary_text,
                                'timestamp': firestore.SERVER_TIMESTAMP
                            })
                            st.success("Đã lưu vào Lịch sử.")
                        except Exception as e:
                            st.warning(f"Không thể lưu vào Firebase: {e}")
                    else:
                        st.warning("Không thể lưu: Firebase chưa được khởi tạo.")


    with tab2:
        st.header("Lịch sử Gợi ý")
        # Hiển thị lịch sử (truy vấn Firestore)
        if st.session_state.db: # Kiểm tra lần nữa trước khi truy vấn
            try:
                itineraries_ref = st.session_state.db.collection('itineraries').where('user_id', '==', st.session_state.user_id).order_by('timestamp', direction=firestore.Query.DESCENDING).limit(10)

                for doc in itineraries_ref.stream():
                    data = doc.to_dict()
                    st.info(f"📍 **{data['destination']}** - Tốc độ: {data['input']['pace']}")
                    st.caption(f"Ngày: {data['input']['dates']} | Sở thích: {', '.join(data['input']['interests'])}")
                    st.markdown(data['itinerary'])
                    st.divider()
            except Exception as e:
                st.error(f"Không thể tải lịch sử từ Firebase: {e}")
        else:
            st.warning("Không thể hiển thị lịch sử: Firebase chưa được khởi tạo.")

In [ ]:
# Tắt tunnel cũ (Ollama)
from pyngrok import ngrok
# ngrok.kill()

# TẢI LẠI OLLAMA URL ĐÃ TẠO Ở BƯỚC 1 (Cell 3) VÀ DÁN VÀO FILE app.py (Cell 4) TRƯỚC KHI CHẠY CELL NÀY

# Tạo tunnel mới cho Streamlit (Port 8501)
streamlit_tunnel = ngrok.connect(8501, proto="http")
print("Streamlit Public URL:", streamlit_tunnel.public_url)

# Khởi chạy Streamlit App ở chế độ nền
# Dùng &>/dev/null& để ẩn output và chạy ngầm
!streamlit run app.py &>/dev/null&

print("Bây giờ bạn có thể truy cập Streamlit Public URL để sử dụng ứng dụng.")